# Creates a file called swapped_coords.xyz that swaps the coordinates to our desired outcome. 
# Requires polymers.txt (the initial atomic positions) in same directory. 

In [200]:
import numpy as np
names = 'index,chain,type,xcoord,ycoord,zcoord'
coords = np.genfromtxt('polymers.txt', delimiter = ',', names = names, dtype = float)
coords[0][0] = 1 # somehow the first became a Nan? So I reset

In [201]:
CHAIN_LENGTH = 20 # number of chains per polymer
STIFFENED_CHAINS = 20 # number of chains to stiffen
TOTAL_CHAINS = 400 # total number of chains
TYPE_1 = 1 # atom type 1
TYPE_2 = 2 # atom_type 2

# we want a new chain every *this* many steps:
chain_steps = TOTAL_CHAINS/STIFFENED_CHAINS 

def iofc(chain):
    # "index of chain": takes chain number and returns first possible index
    return int(CHAIN_LENGTH *(chain - 1) + 1)

# inverse of the above, for debugging
def cofi(index):
    return int((index + 19)/CHAIN_LENGTH)

def swap_chain(chain):
    # big picture is that 20, 40 ...380 -> 381, 382...399
    return chain//20 + int(TOTAL_CHAINS - STIFFENED_CHAINS)

In [202]:
for j in range(1, STIFFENED_CHAINS): 
    current_chain = CHAIN_LENGTH*j # the number of the chain we want to move to the end
    new_chain = swap_chain(current_chain) # the position of the moved chain
    for i in range((iofc(current_chain) - 1), iofc(current_chain + 1) -1):
        new_index = iofc(new_chain) - iofc(current_chain) + (i + 1)
        coords[i]['index'] = int(new_index) # update the chain index
        coords[i]['chain'] = int(new_chain) # change the chain
        coords[i]['type'] = int(TYPE_2) # change the type
    for i in range((iofc(current_chain + 1) -1), iofc(current_chain + chain_steps) -1):
        coords[i]['index'] -= CHAIN_LENGTH *j 
        coords[i]['chain'] -=j
        # we don't modify the atom type here  
        
 # this loop does not catch the final case, 
# where the atoms on the final chain become of type 2. 
# but it's getting late and I will fix it in an inelegant way:

# naive method to convert this to a 2d array
# todo: investigate more pythonic ways of doing this
x = []
for arrs in coords:
    items = []
    for item in arrs: 
        items.append(item)
    x.append(items)
x = np.array(x)
# if chain == 400, make type = 2
x[:,2][x[:,1] == 400] = TYPE_2

In [203]:
np.savetxt('swapped_polys.xyz', x, delimiter = ' ')